# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd

In [2]:
# load data from database
from sqlalchemy import create_engine
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql('SELECT * FROM InsertTableName', con=engine)
df.head()

,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,direct,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,direct,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,direct,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
#Define feature and target variables X and Y
import numpy as np
X = df['message']
y = df.iloc[:, 3:]
y['related'].replace(2, 1, inplace=True)

### 2. Write a tokenization function to process your text data

In [4]:
import nltk
import re
nltk.download(['punkt', 'stopwords','wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
def tokenize(text):
    text = re.sub(r'[^\w\s]','',text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tok = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in stopwords.words("english")]

    return clean_tok

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [5]:
for message in X[:5]:
    print(message)
    print(tokenize(message))

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']
Is the Hurricane over or is it not over
['hurricane']
Looking for someone but no name
['looking', 'someone', 'name']
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '8090', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately']
says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)), 
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
from sklearn.metrics import classification_report

y_pred = pipeline.predict(X_test)

for i in range(35):
    print("Column: {}".format(y_test.columns[i]))
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

Column: related
             precision    recall  f1-score   support

        0.0       0.28      0.12      0.17      1861
        1.0       0.77      0.90      0.83      6002

avg / total       0.65      0.72      0.67      7863

Column: request
             precision    recall  f1-score   support

        0.0       0.84      0.97      0.90      6555
        1.0       0.36      0.08      0.14      1308

avg / total       0.76      0.82      0.77      7863

Column: offer
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      7831
        1.0       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      7863

Column: aid_related
             precision    recall  f1-score   support

        0.0       0.60      0.77      0.67      4660
        1.0       0.42      0.24      0.31      3203

avg / total       0.53      0.56      0.53      7863

Column: medical_help
             precision    recall  f1-score   support


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f8db82bc8c8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [11]:
from sklearn.model_selection import GridSearchCV

parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'vect__max_df': [0.75, 1.0],
              'clf__estimator__verbose': [3]}

cv = GridSearchCV(pipeline, param_grid = parameters, cv = 2)

model = cv.fit(X_train, y_train)
y_pred = model.predict(X_test)

building tree 1 of 10


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s


building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


building tree 1 of 10
building tree 2 of 10


...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


building tree 2 of 10


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.1s remaining:    0.0s


building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    5.5s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(

In [23]:
print('Best Parameters: {}'.format(model.best_params_))

Best Parameters: {'clf__estimator__verbose': 3, 'vect__max_df': 1.0, 'vect__ngram_range': (1, 1)}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [29]:
print('Accuracy:')
print((y_pred == y_test).mean())
for i in range(35):
    print("Column: {}".format(y_test.columns[i]))
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

Accuracy:
related                   0.718682
request                   0.824240
offer                     0.995803
aid_related               0.560600
medical_help              0.917080
medical_products          0.948239
search_and_rescue         0.972784
security                  0.979143
military                  0.966171
child_alone               1.000000
water                     0.933613
food                      0.887320
shelter                   0.902073
clothing                  0.984103
money                     0.975455
missing_people            0.987409
refugees                  0.960193
death                     0.951927
other_aid                 0.854381
infrastructure_related    0.928526
transport                 0.949638
buildings                 0.943279
electricity               0.979652
tools                     0.994659
hospitals                 0.988681
shops                     0.995167
aid_centers               0.987664
other_infrastructure      0.954216
weather_re

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [39]:
from sklearn.ensemble import AdaBoostClassifier

pipeline = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultiOutputClassifier(AdaBoostClassifier()))])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print('Accuracy:')
print((y_pred == y_test).mean())
for i in range (35):
    print("Column: {}".format(y_test.columns[i]))
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

Accuracy:
related                   0.761796
request                   0.825130
offer                     0.995294
aid_related               0.585273
medical_help              0.922040
medical_products          0.950655
search_and_rescue         0.973165
security                  0.979270
military                  0.965789
child_alone               1.000000
water                     0.939209
food                      0.892407
shelter                   0.908941
clothing                  0.982831
money                     0.975455
missing_people            0.986646
refugees                  0.960575
death                     0.953199
other_aid                 0.871932
infrastructure_related    0.933995
transport                 0.951037
buildings                 0.947984
electricity               0.979397
tools                     0.994404
hospitals                 0.987282
shops                     0.994277
aid_centers               0.987537
other_infrastructure      0.955106
weather_re

### 9. Export your model as a pickle file

In [41]:
import pickle
filename = 'finalized_model.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.